In [ ]:
import os
import shutil
import pathlib
import numpy as np
import pandas as pd

run_id = 1
main_folder_path = pathlib.Path("/home/alp/noetic_ws/src/simulation/images")
folder_path = main_folder_path / f"run_{run_id}"
csv_path = folder_path / "cargo_data.csv"
copy_similar = True  # Changed from delete_similar to copy_similar

# Create output folder for non-recurrent frames
output_folder = main_folder_path / "non_recurrent"
output_csv_path = main_folder_path / "cargo_data.csv"

def normalize(q):
    return q / np.linalg.norm(q)

def quat_angle_diff(q1, q2):
    dot = np.clip(np.dot(q1, q2), -1.0, 1.0)
    angle_rad = 2 * np.arccos(abs(dot))
    return np.degrees(angle_rad)

# Read the CSV file
df = pd.read_csv(csv_path)

# Normalize quaternions
quats = df[['cargo_rot_x', 'cargo_rot_y', 'cargo_rot_z', 'cargo_rot_w']].apply(lambda row: normalize(row.values), axis=1)

# Reference quaternion (identity quaternion)
ref_q = normalize(np.array([0.0, 0.0, 0.0, 1.0]))

# Thresholds
abs_threshold = 5.0
seq_threshold = 2.0

# Counters
abs_close_count = 0
seq_close_count = 0

# Lists to track what to copy (keep the diverse/representative frames)
files_to_copy = []
frame_ids_to_keep = set()

i = 0
while i < len(df):
    q = quats[i]
    frame_id = int(df.loc[i, 'frameId'])
    
    # Check if close to reference quaternion
    angle_from_ref = quat_angle_diff(ref_q, q)
    
    if angle_from_ref < abs_threshold:
        abs_close_count += 1
        # Skip this frame entirely (too close to reference)
        i += 1
        continue
    
    # This frame is good - add it to keep list
    fname = f"{frame_id}.jpg"
    files_to_copy.append(fname)
    frame_ids_to_keep.add(frame_id)
    
    # Now check how many subsequent frames are similar to THIS frame (not to each other)
    base_q = q  # This is our reference frame for the sequence
    j = i + 1
    
    while j < len(df):
        next_q = quats[j]
        angle_from_base = quat_angle_diff(base_q, next_q)  # Compare to the base frame, not previous
        
        if angle_from_base < seq_threshold:
            seq_close_count += 1
            j += 1  # Skip this similar frame
        else:
            break  # Found a different frame, stop the sequence
    
    # Move index to the next different frame
    i = j

if copy_similar:
    # Create output directory if it doesn't exist
    output_folder.mkdir(exist_ok=True)
    
    # Copy files to the new folder
    copied_count = 0
    for file_name in files_to_copy:
        source_path = folder_path / file_name
        dest_path = output_folder / file_name
        
        if os.path.exists(source_path):
            shutil.copy2(source_path, dest_path)
            copied_count += 1
        else:
            print(f"Warning: {file_name} not found in source folder")
    
    # Create new CSV with only the frames we're keeping (diverse/representative ones)
    # Filter dataframe to keep only the frames we identified as diverse
    filtered_df = df[df['frameId'].isin(frame_ids_to_keep)]
    
    # Sort by frameId to maintain sequentiality
    filtered_df = filtered_df.sort_values('frameId').reset_index(drop=True)
    
    # Save the filtered CSV
    filtered_df.to_csv(output_csv_path, index=False)
    
    print(f"Created output folder: {output_folder}")
    print(f"Copied {copied_count} image files")
    print(f"Created new CSV with {len(filtered_df)} rows")

print(f"Original total frames: {len(df)}")
print(f"Total close-to-zero frames: {abs_close_count}")
print(f"Total sequentially close frames: {seq_close_count}")
print(f"Total frames to copy (diverse representatives): {len(files_to_copy)}")

if copy_similar:
    print(f"Diverse/representative frames saved to: {output_folder}")
    print(f"New CSV contains {len(filtered_df)} frames (maintaining sequential order)")
else:
    print(f"Would copy {len(files_to_copy)} diverse frames to separate folder")

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Subset
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
import matplotlib.pyplot as plt
import csv
from datetime import datetime

from model import StateBasedModel
from dataloader import StateBasedDroneDataset

pl.seed_everything(42)

CONFIG = {
    "base_folder": "/home/alp/noetic_ws/src/simulation/images/run_8",
    "sequence_length": 10,
    "hidden_dim": 256,
    "state_dim": 128,
    "dropout_rate": 0.3,
    "λ_rot": 0.8,
    "λ_dir": 1.5,
    "λ_dist": 0.1,
    "λ_pos": 1.5,
    "λ_vel": 0.2,
    "λ_temporal": 0.3,
    "λ_state_reg": 0.1,
    "batch_size": 16,
    "learning_rate": 1e-4,
    "num_epochs": 100,
    "num_workers": 8,
    "gradient_clip": 1.0,
    "val_check_interval": 0.5,
    "early_stop_patience": 15,
}

def create_datasets():
    dataset = StateBasedDroneDataset(
        images_folder=CONFIG["base_folder"],
        csv_path=os.path.join(CONFIG["base_folder"], "cargo_data.csv"),
        sequence_length=CONFIG["sequence_length"],
        augment=False,
        start_from_zero_state=True
    )
    
    total_sequences = len(dataset)
    n_train_seq = int(0.7 * total_sequences)
    n_val_seq = int(0.15 * total_sequences)
    
    train_indices = list(range(0, n_train_seq))
    val_indices = list(range(n_train_seq, n_train_seq + n_val_seq))
    test_indices = list(range(n_train_seq + n_val_seq, total_sequences))
    
    train_dataset = Subset(dataset, train_indices)
    val_dataset = Subset(dataset, val_indices)
    test_dataset = Subset(dataset, test_indices)
    
    dataset.augment = True
    
    print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")
    return train_dataset, val_dataset, test_dataset

def create_data_loaders(train_dataset, val_dataset, test_dataset):
    train_loader = DataLoader(
        train_dataset, batch_size=CONFIG["batch_size"], shuffle=False,
        num_workers=CONFIG["num_workers"], pin_memory=True, drop_last=True, persistent_workers=True
    )
    val_loader = DataLoader(
        val_dataset, batch_size=CONFIG["batch_size"], shuffle=False,
        num_workers=CONFIG["num_workers"], pin_memory=True, persistent_workers=True
    )
    test_loader = DataLoader(
        test_dataset, batch_size=CONFIG["batch_size"], shuffle=False,
        num_workers=CONFIG["num_workers"], pin_memory=True, persistent_workers=True
    )
    return train_loader, val_loader, test_loader

def create_model():
    return StateBasedModel(
        λ_rot=CONFIG["λ_rot"], λ_dir=CONFIG["λ_dir"], λ_dist=CONFIG["λ_dist"],
        λ_pos=CONFIG["λ_pos"], λ_vel=CONFIG["λ_vel"], λ_temporal=CONFIG["λ_temporal"],
        λ_state_reg=CONFIG["λ_state_reg"], hidden_dim=CONFIG["hidden_dim"],
        state_dim=CONFIG["state_dim"], dropout_rate=CONFIG["dropout_rate"]
    )

def setup_trainer():
    callbacks = [
        ModelCheckpoint(
            monitor='val_loss', mode='min', save_top_k=3,
            filename='state_model-{epoch:02d}-{val_loss:.4f}',
            dirpath='./checkpoints/state_based_model', save_last=True
        ),
        EarlyStopping(monitor='val_loss', patience=CONFIG["early_stop_patience"], mode='min', verbose=True),
        LearningRateMonitor(logging_interval='epoch')
    ]
    
    logger = TensorBoardLogger(save_dir='lightning_logs', name='state_based_cargo_tracking', version=None)
    
    return pl.Trainer(
        logger=logger, callbacks=callbacks, max_epochs=CONFIG["num_epochs"],
        accelerator='gpu' if torch.cuda.is_available() else 'cpu', devices=1,
        precision='16-mixed' if torch.cuda.is_available() else '32',
        gradient_clip_val=CONFIG["gradient_clip"], val_check_interval=CONFIG["val_check_interval"],
        log_every_n_steps=10, accumulate_grad_batches=2, deterministic=True
    )

def calculate_errors(pred_numpy, label_numpy):
    """Calculate comprehensive errors"""
    pred_dir = pred_numpy[:3]
    pred_dist = pred_numpy[3]
    pred_rot = pred_numpy[4:8]
    pred_vel = pred_numpy[8:11]
    
    gt_dir = label_numpy[:3]
    gt_dist = label_numpy[3]
    gt_rot = label_numpy[4:8]
    gt_vel = label_numpy[8:11]
    
    # Direction error
    pred_dir_norm = pred_dir / (np.linalg.norm(pred_dir) + 1e-8)
    gt_dir_norm = gt_dir / (np.linalg.norm(gt_dir) + 1e-8)
    cos_sim = np.clip(np.dot(pred_dir_norm, gt_dir_norm), -1.0, 1.0)
    direction_error = np.degrees(np.arccos(np.abs(cos_sim)))
    
    # Distance errors
    distance_error_abs = abs(pred_dist - gt_dist)
    distance_error_rel = distance_error_abs / max(abs(gt_dist), 1e-6) * 100
    
    # Rotation error
    pred_rot_norm = pred_rot / (np.linalg.norm(pred_rot) + 1e-8)
    gt_rot_norm = gt_rot / (np.linalg.norm(gt_rot) + 1e-8)
    dot_product = np.abs(np.dot(pred_rot_norm, gt_rot_norm))
    rotation_error = 2 * np.degrees(np.arccos(np.clip(dot_product, 0, 1)))
    
    # Velocity error
    velocity_error = np.linalg.norm(pred_vel - gt_vel)
    
    # Position error
    pred_pos = pred_dir_norm * pred_dist
    gt_pos = gt_dir_norm * gt_dist
    position_error = np.linalg.norm(pred_pos - gt_pos)
    
    return {
        'direction_error_degrees': direction_error,
        'distance_error_abs': distance_error_abs,
        'distance_error_rel': distance_error_rel,
        'rotation_error_degrees': rotation_error,
        'velocity_error_magnitude': velocity_error,
        'position_error_magnitude': position_error
    }

def analyze_test_set(model, test_loader):
    """Analyze test set predictions over time and save to CSV"""
    print("Analyzing test set predictions...")
    
    # Create CSV file
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    csv_filename = f"test_predictions_{timestamp}.csv"
    
    headers = [
        'batch_idx', 'sample_idx', 'frame_idx', 'sequence_time',
        'pred_dir_x', 'pred_dir_y', 'pred_dir_z', 'gt_dir_x', 'gt_dir_y', 'gt_dir_z',
        'pred_distance', 'gt_distance',
        'pred_rot_x', 'pred_rot_y', 'pred_rot_z', 'pred_rot_w',
        'gt_rot_x', 'gt_rot_y', 'gt_rot_z', 'gt_rot_w',
        'pred_vel_x', 'pred_vel_y', 'pred_vel_z', 'gt_vel_x', 'gt_vel_y', 'gt_vel_z',
        'direction_error_degrees', 'distance_error_abs', 'distance_error_rel',
        'rotation_error_degrees', 'velocity_error_magnitude', 'position_error_magnitude'
    ]
    
    with open(csv_filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(headers)
        
        model.eval()
        total_samples = 0
        
        with torch.no_grad():
            for batch_idx, batch in enumerate(test_loader):
                images = batch['image_sequence'].to(model.device)
                labels = batch['label_sequence'].to(model.device)
                
                # Get predictions for entire sequence
                predictions, _, _ = model(images, return_all_states=True)
                
                # Process each sample and frame in the batch
                batch_size, seq_len = images.shape[0], images.shape[1]
                
                for sample_idx in range(batch_size):
                    for frame_idx in range(seq_len):
                        pred = predictions[sample_idx, frame_idx].cpu().numpy()
                        gt = labels[sample_idx, frame_idx].cpu().numpy()
                        
                        # Calculate errors
                        errors = calculate_errors(pred, gt)
                        
                        # Prepare CSV row
                        csv_row = [
                            batch_idx, sample_idx, frame_idx, frame_idx * 0.1,  # assuming 10fps
                            # Predictions and ground truth
                            pred[0], pred[1], pred[2], gt[0], gt[1], gt[2],  # direction
                            pred[3], gt[3],  # distance
                            pred[4], pred[5], pred[6], pred[7], gt[4], gt[5], gt[6], gt[7],  # rotation
                            pred[8], pred[9], pred[10], gt[8], gt[9], gt[10],  # velocity
                            # Errors
                            errors['direction_error_degrees'], errors['distance_error_abs'],
                            errors['distance_error_rel'], errors['rotation_error_degrees'],
                            errors['velocity_error_magnitude'], errors['position_error_magnitude']
                        ]
                        
                        writer.writerow(csv_row)
                        total_samples += 1
                
                if batch_idx % 10 == 0:
                    print(f"Processed batch {batch_idx}/{len(test_loader)}")
    
    print(f"Test analysis complete. Saved {total_samples} predictions to {csv_filename}")
    return csv_filename

def plot_test_analysis(csv_filename):
    """Generate time series plots for test predictions"""
    try:
        df = pd.read_csv(csv_filename)
        print(f"Loaded {len(df)} test predictions from {csv_filename}")
        
        # Create time index (continuous time across all sequences)
        df['global_time'] = df['batch_idx'] * CONFIG["sequence_length"] * 0.1 + df['sequence_time']
        
        # Create plots
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        axes = axes.flatten()
        
        error_configs = [
            ('direction_error_degrees', 'Direction Error (°)', 'blue'),
            ('distance_error_abs', 'Distance Error (m)', 'red'),
            ('distance_error_rel', 'Distance Error (%)', 'orange'),
            ('rotation_error_degrees', 'Rotation Error (°)', 'green'),
            ('velocity_error_magnitude', 'Velocity Error (m/s)', 'purple'),
            ('position_error_magnitude', 'Position Error (m)', 'brown')
        ]
        
        for idx, (col, title, color) in enumerate(error_configs):
            ax = axes[idx]
            times = df['global_time'].values
            errors = df[col].values
            
            # Plot with transparency to show density
            ax.plot(times, errors, color=color, linewidth=0.5, alpha=0.6)
            
            # Add rolling average
            if len(errors) > 100:
                window = min(100, len(errors) // 10)
                rolling_mean = pd.Series(errors).rolling(window=window, center=True).mean()
                ax.plot(times, rolling_mean, color='darkred', linewidth=2, label=f'Rolling Mean (w={window})')
            
            ax.set_title(title, fontsize=14, fontweight='bold')
            ax.set_xlabel('Time (seconds)')
            ax.set_ylabel('Error')
            ax.grid(True, alpha=0.3)
            
            # Add statistics
            mean_error = np.mean(errors)
            ax.axhline(y=mean_error, color='red', linestyle='--', alpha=0.7, 
                      label=f'Mean: {mean_error:.3f}')
            if len(errors) > 100:
                ax.legend()
        
        plt.tight_layout()
        plot_filename = csv_filename.replace('.csv', '_plots.png')
        plt.savefig(plot_filename, dpi=300, bbox_inches='tight')
        print(f"Test analysis plots saved to: {plot_filename}")
        plt.show()
        
        # Print summary statistics
        print(f"\n{'='*60}")
        print("TEST SET ANALYSIS SUMMARY")
        print(f"{'='*60}")
        print(f"Total predictions: {len(df)}")
        print(f"Total time span: {df['global_time'].max():.1f} seconds")
        print(f"Unique sequences: {df['batch_idx'].nunique()}")
        
        print(f"\n{'Metric':<25} {'Mean':<10} {'Std':<10} {'Max':<10}")
        print("-" * 55)
        
        for col, title, _ in error_configs:
            values = df[col].values
            mean_val = np.mean(values)
            std_val = np.std(values)
            max_val = np.max(values)
            print(f"{title:<25} {mean_val:<10.3f} {std_val:<10.3f} {max_val:<10.3f}")
        
    except Exception as e:
        print(f"Error in test analysis: {e}")

def main():
    print("Starting training...")
    
    # Create datasets and loaders
    train_dataset, val_dataset, test_dataset = create_datasets()
    train_loader, val_loader, test_loader = create_data_loaders(train_dataset, val_dataset, test_dataset)
    
    # Create model and trainer
    model = create_model()
    trainer = setup_trainer()
    
    print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    # Train
    trainer.fit(model, train_loader, val_loader)
    
    # Test
    print("Evaluating on test set...")
    test_results = trainer.test(model, test_loader, ckpt_path='best')
    
    for metric, value in test_results[0].items():
        if 'loss' in metric:
            print(f"  {metric}: {value:.4f}")
        elif 'degrees' in metric:
            print(f"  {metric}: {value:.2f}°")
        else:
            print(f"  {metric}: {value:.4f}")
    
    # Load best model for detailed test analysis
    best_model_path = trainer.checkpoint_callback.best_model_path
    model = StateBasedModel.load_from_checkpoint(best_model_path)
    model.eval()
    model.to('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Analyze test set predictions over time
    csv_filename = analyze_test_set(model, test_loader)
    plot_test_analysis(csv_filename)
    
    # Save model
    torch.save(model.state_dict(), "state_based_weights.pth")
    trainer.save_checkpoint("state_based_model_final.ckpt")
    print("Model saved successfully!")

if __name__ == "__main__":
    main()

In [ ]:
test_results = trainer.test(model, test_loader)


In [ ]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Subset
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from dataloader import DroneDataset
from model import Model

torch.set_float32_matmul_precision('high')

SEQUENCE_LENGTH = 5  # Number of consecutive frames to use
BATCH_SIZE = 8
NUM_WORKERS = 8

 = 1e-4
FPS = 10  # Your video frame rate

full_dataset = DroneDataset(
    images_folder="./data/images",
    csv_path="./data/cargo_data.csv",
    sequence_length=SEQUENCE_LENGTH,
)

total_size = len(full_dataset)
train_end = int(0.7 * total_size)
val_end = train_end + int(0.15 * total_size)
test_indices = list(range(val_end, total_size))
test_dataset = Subset(full_dataset, test_indices)


test_loader = DataLoader(
    test_dataset,  # Using full dataset for testing like in original
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=NUM_WORKERS,
    pin_memory=True
)

# Model
model = Model(
    λ_rot=0.8,    # Rotation loss weight
    λ_dir=0.5,    # Direction vector loss weight
    λ_dist=0.1,   # Distance loss weight
    λ_pos=1.5,    # Position loss weight
    λ_vel=0.3,    # Velocity loss weight
    sequence_length=SEQUENCE_LENGTH
)

# Create trainer
trainer = pl.Trainer()

# Load weights from checkpoint file
checkpoint_path = "weights.ckpt"
model = Model.load_from_checkpoint(checkpoint_path)

# Test the model using the loaded weights
trainer.test(model, dataloaders=test_loader)